In [1]:
from scipy.stats import gamma
from scipy.stats import norm as normal
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots


In [2]:
%%time
rng = np.random.default_rng() # PCG64 generator

x = np.array([576, 635, 558, 578, 666, 580, 555, 661, 651, 605, 653, 575, 545, 572, 594])
y = np.array([3.39, 3.3, 2.81, 3.03, 3.44, 3.07, 3, 3.43, 3.36, 3.13, 3.12, 2.74, 2.76, 2.88, 2.96])

beta0, beta1, tau = 0, 0, 1
n_iter = 10000
n_burn_in = 1000

Wall time: 0 ns


In [3]:
def gibbs(x,y, beta0, beta1, tau, n_iter, n_burn_in):
    n = len(x) 
    # https://stackoverflow.com/questions/35213592/numpy-calculate-square-of-norm-2-of-vector
    sqnorm = lambda x: np.inner(x, x)
    xx = sqnorm(x)
    a = n / 2

    beta0_s, beta1_s, tau_s = [], [], []
    for iter in range(0, n_iter):
        # Sample t ~ Ga (alpha = n/2, beta = 0.5 * sum (y - b0 - b1*x)**2)
        one_over_b = 2 / sqnorm(y - beta0 - beta1 * x)
        tau = gamma.rvs(a, scale  = one_over_b)

        # Sample beta0 ~ N (mu = sum(y - b1 * x)/n,  sigma = sqrt(1/(tau*n)))
        mu = np.average(y - beta1 * x)
        sigma = np.sqrt(1.0/ (tau * n))
        beta0 = normal.rvs(loc = mu, scale = sigma)

        # Sample beta0 ~ N (mu = sum((y - b0) * x)/sum(x**2),  sigma = sqrt(1/(tau*sum(x**2))))
        mu = np.sum((y - beta0) * x) / xx
        sigma = np.sqrt(1.0 / (tau * xx) )
        beta1 = normal.rvs(loc = mu, scale = sigma)

        # Record values
        if iter > n_burn_in:
            beta0_s.append(beta0)
            beta1_s.append(beta1)
            tau_s.append(tau)
    return beta0_s, beta1_s, tau_s

In [4]:
%%time
beta0_s, beta1_s, tau_s = gibbs(x,y, beta0, beta1, tau, n_iter, n_burn_in)

Wall time: 1.3 s


In [5]:
def vis(n_iter, beta0_s, beta1_s, tau_s):
    fig1 = make_subplots(specs=[[{"secondary_y": True}]])
    n_last = 500
    fig1.add_trace(px.scatter(x=range(n_iter-n_last, n_iter), y=beta0_s[-n_last:], 
                              labels={"x": "round", "y": "<i>β<sub>0</sub></i>"}).data[0], 
                   secondary_y=False)
    fig1.add_trace(px.scatter(x=range(n_iter-n_last, n_iter), y=beta1_s[-n_last:],
                              labels={"x": "round", "y": "<i>β<sub>1</sub></i>"},
                              color_discrete_sequence=['red']).data[0], secondary_y=True)
    fig1.update_layout(title_text=f"last {n_last} Gibbs rounds of <i>β<sub>0,1</sub></i>")
    fig1.update_xaxes(title_text="round")
    fig1.update_yaxes(title_text="<i>β<sub>0</sub></i>", secondary_y=False)
    fig1.update_yaxes(title_text="<i>β<sub>1</sub></i>", secondary_y=True)

    fig1.show("notebook_connected")

    fig2 = px.scatter(x=range(n_iter-n_last, n_iter), y=tau_s[-n_last:], 
                      labels={"x": "round", "y": "<i>τ</i>"},
                      title=f"last {n_last} Gibbs rounds of <i>τ</i>"
                     )
    fig2.show("notebook_connected")

In [6]:
vis(n_iter, beta0_s, beta1_s, tau_s)

In [7]:
%%time
x_p = x - np.average(x)
beta0_p_s, beta1_p_s, tau_p_s = gibbs(x_p,y, beta0, beta1, tau, n_iter, n_burn_in)
vis(n_iter, beta0_p_s, beta1_p_s, tau_p_s)

Wall time: 1.55 s
